# HydroSOS Publish Status Products
#### Jose Valles (jose.valles.leon@gmail.com)

In [1]:
import os
import requests
import json 

# This code line allows to remove warning produce by verify = False
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Send the results to Dinagua Web Service for publishing

After creating the json file for the corresponding year-month, we will update the Web Service using the following process

First, we are going to obtain the token

In [2]:
# Ambiente produccion
url_base = 'https://www.ambiente.gub.uy/dinaguaws/'
# Username and password
user_dinagua = {"user":"FEWS-Uruguay","password":"4MEzOeKFgp0pj9lATSIF"}
# Get the token from DINAGUA WS
response_token = requests.post(url=url_base + "gettoken", json=user_dinagua, verify=False);
response_token.close()
# Extract token and assign it to a variable
token = response_token.text
# Add the token to a bearer authorization
headers = {"Authorization" : "Bearer " + token}

Define the parameters for publishing the status results

In [18]:
# Locate the corresponding json file
json_file = '../output_json/01_month/2024-08.json'
# json_file = '../waterbalance/output_json/12_month/2024-08.json'
date_name = json_file.split('/')[-1].split('.')[0]
# scale: 1, 3, 12
cat_scale = 1
# ts: Estaciones or Modelos
ts = "Estaciones"

POST Method for creating the filter Id

In [19]:
# create the json dictonary
dataFiltro = {"fecha": date_name,"temporalidad": cat_scale,"serietemporal": ts}

# POST Save Filter
post_filtro = requests.post(url=url_base + "estadohidro/datoscat/guardarFiltro", headers = headers, json=dataFiltro, verify=False)
post_filtro.close()
print(post_filtro.text)

Filtro guardado con éxito


GET Method for extracting the FilterID

In [20]:
# Create the json dictonary
params_filtros = {"fecha" : date_name, "temporalidad" : cat_scale, "serietemporal" : ts}
# Send requests
get_filtro_id = requests.get(url = url_base + "estadohidro/filtros",headers=headers, params=params_filtros, verify=False)
get_filtro_id.close()
# print id
id = json.loads(get_filtro_id.content.decode('utf-8'))
id = id[0]["id"]
print(id)

30


POST HydroSOS Status for the corresponding Month/Year

In [21]:
# Creating the Request Body
datosCat = {"filtrosId": {id}}
# importing the json file from the output directory
json_body = open(json_file)
json_body = json.load(json_body)
# POST HydroSOS status 
post_estado = requests.post(url=url_base+"estadohidro/datoscat/guardarDatosCat",headers=headers,params=datosCat,json=json_body,verify=False)
post_estado.close()
print(post_estado.text)

DatosCat guardado con éxito


We are going to see if the hydrological status is correctly added in the service

In [22]:
params_status = {"serietemporal": ts, "fecha": date_name, "temporalidad": cat_scale}
get_status = requests.get(url=url_base+"estadohidro/datoscat",headers=headers,params=params_status,verify=False)
get_status.close()
# Convert it to json and dataframe
response_status = json.loads(get_status.content.decode('utf-8'))
response_status

[{'id': 859, 'category': 3, 'stationID': '520', 'filtros_id': 30},
 {'id': 860, 'category': 3, 'stationID': '22060', 'filtros_id': 30},
 {'id': 861, 'category': 3, 'stationID': '101', 'filtros_id': 30},
 {'id': 862, 'category': 2, 'stationID': '440', 'filtros_id': 30},
 {'id': 863, 'category': 2, 'stationID': '1330', 'filtros_id': 30},
 {'id': 864, 'category': 4, 'stationID': '1230', 'filtros_id': 30},
 {'id': 865, 'category': 2, 'stationID': '1170', 'filtros_id': 30},
 {'id': 866, 'category': 2, 'stationID': '150', 'filtros_id': 30},
 {'id': 867, 'category': 3, 'stationID': '970', 'filtros_id': 30}]